In [14]:
from PIL import Image
import os, glob

import tensorflow as tf
from tensorflow import keras
import numpy as np

import matplotlib.pyplot as plt
#쪼굼 알거 갔네

In [15]:
def load_data(img_path,source):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=300   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print(source,"의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/workplace/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path,"train")
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

image_dir_path_1 = os.getenv("HOME") + "/workplace/rock_scissor_paper_1"
(x_test, y_test)=load_data(image_dir_path_1,"test")
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화


train 의 이미지 개수는 300 입니다.
test 의 이미지 개수는 300 입니다.


In [16]:
n_channel_1=256
n_channel_2=512
n_dense=64

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [17]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

Epoch 1/10
10/10 [==============================] - 0s 7ms/step - loss: 68.0927 - accuracy: 0.1967
Epoch 2/10
10/10 [==============================] - 0s 7ms/step - loss: 2.8761 - accuracy: 0.3500
Epoch 3/10
10/10 [==============================] - 0s 6ms/step - loss: 0.9400 - accuracy: 0.5933
Epoch 4/10
10/10 [==============================] - 0s 5ms/step - loss: 0.2953 - accuracy: 0.9500
Epoch 5/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0754 - accuracy: 0.9933
Epoch 6/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0189 - accuracy: 1.0000
Epoch 7/10
10/10 [==============================] - 0s 6ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 8/10
10/10 [==============================] - 0s 6ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 9/10
10/10 [==============================] - 0s 6ms/step - loss: 8.3804e-04 - accuracy: 1.0000
Epoch 10/10
10/10 [==============================] - 0s 6ms/step - loss: 4.8939e-04 - accuracy: 1.0000


In [18]:
test_loss, test_accuracy = model.evaluate(x_test,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 4.3606 - accuracy: 0.3100
test_loss: 4.360597610473633 
test_accuracy: 0.3100000023841858


train : 본인 작성 300개,
test  : 타인 작성 300개

    mnist 에 비해 정확도가 떨어지는 사항은
        1. 흑백이 아닌 컬러
        2. 배경이 깨끗한 사항이 아니라 오브젝트 인식의 어려움이 존재함
        3. 정확도를 올리기엔 부족한 데이터수
        
데이터를 여럿 모아서 훈련시켜 정확도를 올리는 작업도 진행 해보았으나 금번 제출본에서는 하기 파라미터 설정만으로  확인했다.

n_channel_1=256, 
n_channel_2=512, 
n_dense=64
